# Análise comparativa de modelos

Esse notebook destina-se a uma análise comparativa de diferentes abordagens para predição de sentimento em tweets. O objetivo final é analisar diferentes combinações de modelos, vetorizadores e normalizadores e seus respectivos hiperparâmetros para definir dentro todas as combinações possíveis aquela que tenha uma melhor desempenho geral. Para garantir isso será efetuada uma validação cruzada para cada combinação possível tanto de modelos, quanto de hiperparâmetros.

## Importando dependências

In [1]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfVectorizer,
)
from sklearn.model_selection import ShuffleSplit, RandomizedSearchCV, cross_validate
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from nltk.tokenize import TweetTokenizer
from sklearn import svm

nltk.download("stopwords")
tweet_tokenizer = TweetTokenizer()


[nltk_data] Downloading package stopwords to /home/marvin-
[nltk_data]     linux/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Importando dados (modificar) 

In [3]:
df = pd.read_csv(
    "../data/raw/NoThemeTweets.csv", usecols=["tweet_text", "sentiment"]
)

df = df.assign(
    number_words=df.tweet_text.apply(lambda x: len(x.split(" "))),
)  # adiciona coluna com número de palavras

df.drop_duplicates(["tweet_text"], inplace=True)  # remove textos duplicados
df.drop(
    df[df.number_words < 5].index, inplace=True
)  # remove tweets com menos de 5 palavras



(785814, 2)
(695548, 3)


In [ ]:
df = pd.read_csv(
    "data/processed", usecols=["tweet_text", "sentiment"]
)

In [5]:

df["tweet_text"] = df["tweet_text"].apply(
    lambda tweet: formatar_texto(texto=tweet)
)  # formata texto do dataframe


In [6]:
corpus_raw = df.tweet_text.to_list()
corpus_steamed = df.tweet_text.apply(lambda x: stemming(texto=x)).to_list()
corpus_lammetized = df.tweet_text.apply(lambda x: lemmatization(text=x)).to_list()

labels = df.sentiment.replace({"Positivo": 1, "Negativo": 0}).to_list()
stop_words = nltk.corpus.stopwords.words("portuguese") + ["https"] + ["co"]



In [7]:
corpus_steamed[:10]

corpus = {
    "corpus_raw": {
        "corpus_data": corpus_raw,
    },
    "corpus_steamed": {
        "corpus_data": corpus_steamed,
    },
    "corpus_lammetized": {
        "corpus_data": corpus_lammetized,
    }
    }

### Inicialmente será definido um dicionário para cada uma  das etapas na formação de uma abordagem(pipeline), as etapas são:

!["Exemplo de pipeline"](../Diagrama.png)


Cada modelo, vetorizador e normalizador possui seu objeto e um conjunto de hiperparâmetros associados a ele. Para cada abordagem(vetorizador + normalizador + modelo) será feita uma validação cruzada, para garantir a consistência das métricas. Além disso, para garantir uma competição justa, cada abordagem deve ser otimizada com os melhores hiperparâmetros possíveis, para que todas estejam em sua melhor versão. Em vista disso, também é necessário utilizar uma validação cruzada neles.



In [8]:
models = {
    "KNN": {
        "model_obj": KNeighborsClassifier(),
        "hyperparameters": {
            "n_neighbors": [7, 11, 21],
            "weights": ["uniform", "distance"],
        },
    },
    "SMV": {
        "model_obj": svm.SVC(),
        "hyperparameters": {
            "kernel": ["linear", "rbf"],
            "C": [0.1, 0.5, 1, 5, 10],
        },
    },
    "GaussianNB": {
        "model_obj": GaussianNB(),
        "hyperparameters": {
            "var_smoothing": [
                1e-8,
                1e-6,
                1e-4,
                1e-2,
            ]
        },
    },
}

vectorizers = {
    "TfidfVectorizer": {
        "vectorizer_obj": TfidfVectorizer(),
        "hyperparameters": {
            "max_features": [500, 1000, 2000],
            "analyzer": ["word", "char"],
            "stop_words": [stop_words, None],
            "tokenizer": [tweet_tokenizer.tokenize, None],
        },
    },
    "CountVectorizer": {
        "vectorizer_obj": CountVectorizer(),
        "hyperparameters": {
            "max_features": [500, 1000, 2000],
            "analyzer": ["word", "char"],
            "stop_words": [stop_words, None],
            "tokenizer": [tweet_tokenizer.tokenize, None],
        },
    },
}

normalizers = {
    "TrucatedSVD": {
        "normalizer_obj": TruncatedSVD(),
        "hyperparameters": {"n_components": [3, 5, 10, 15]},
    },
}


# Modelos selecionados:  

- ### KNN(k-nearest neighbors):
   O algoritmo KNN é um dos algoritmos clássicos de aprendizado de máquina, usualmente utilizado como algoritmo de classificação a ideia básica proposta  é que pontos semelhantes se encontra próximos um dos outros. Por se tratar de um algoritmo baseado na comparação de dados já existentes, o KNN é considerado um algoritmo do tipo "preguiçoso" já que basicamente decora os pontos do dataset, ou seja, o conhecimento já está diretamente nos dados e não em uma função preditora. No problema em questão sendo uma classificação binaria(positivo ou negativo) a classe definida será a que tiver mais de 50% dos votos. 
  
- Hiperparâmetros:
    - n_neighbors:
      Número de vizinho próximos a ser analisado. A quantidade de pontos é geralmente definida como um número impar para evitar empates na classificação de um novo dado, após isso a classe com maior número de instâncias será a selecionada.
      
    - weights:
      Define se a métrica utilizada será apenas a quantidade, ou se a distância dos pontos terá um peso.

- ### SVC(Support Vector Classification).
  O SVM funciona tentando criar uma hiperplano que separe linearmente os dados em classes diferentes, por exemplo, caso de uma plano 2d é simplesmente uma linha. O critério inicial para  isso é uma hiperplano é que ele consiga separar perfeitamente todos os dados, no caso de haver mais de um hiperplano que faça essa separação é definido como melhor aquele que maximiza a distância das instâncias de cada classe mais próxima. No caso dos dados não sejam linearmente separáveis a priore o SVM consegue aumentar quantidade de dimensões, tornando as classes separáveis dessa forma. No caso da análise de textos, montamos um vetor que represente aquele texto de alguma forma com n-dimensões para montar o hiperplano.

  - Hiperparâmetros:
    - kernel:
    Kernel utilizado para o aumento da dimensionalidade do modelo. Usualmente para aplicações de NLP o linear costuma ser o melhor
      
    - C:
    Parâmetro de regularização, "afrouxa" o critério de separação para ser possível separar mais facilmente os dados.

- ### Gaussian Naive Bayes
  O algoritmo Gausian Naive bayses consiste em fazer uma inferência baseado em várias curvas gaussianas adquiridas através das características do dataset de treino, onde cada uma delas é utilizada como uma parte para definir a probabilidade um dado ser de uma classe específica. No caso de um problema de NLP cada palavra possui sua curva gaussiana, associada com a probabilidade dela ser de uma classe ou outra. Em uma classificação binaria(positiva ou negativa), por exemplo, pode-se partir da pergunta: "Esse texto é positivo?" o algoritmo irá calcular a contagem de cada palavra presente no texto e repassar para cada curva gaussiana respectiva, no final irá tirar um score, a mesma coia será feita para a pergunta: "Esse texto é negativo?", calculando um novo score. Para a pergunta que obtiver o maior score será definida como a classe daquele novo input.
  - Hiperparâmetros:
    - var_smoothing:
    Porção utilizada da maior variância, influencia diretamente na geração da curva.

# Vetorizadores selecionados:  

- ### CountVectorizer
  Essa abordagem faz a contagem das palavras presente para cada uma das instâncias, no caso dessa aplicação tweets, as possibilidades são definidas baseadas no conjunto de todas as palavras possíveis de todos os tweets, o corpus. No final é gerado um vetor com a contagem de palavras presentes em cada tweet.

- Hiperparâmetros:
  - max_features:
    Define a quantidade máxima de palavras que será mantido a contagem, no caso o algoritmo sempre priorizará as palavras que mais aparecem, pois, elas têm um maior peso para a definição da classe.
  - analyzer:
    Define se o algoritmo irá analisar palavra como features ou palavras.
  - stop_words:
    Define um conjunto de palavras ou não para ser removido dos textos antes da contagem, geralmente é removido palavras que não são relevantes para a análise.
  - tokenizer:
    Define o critério usado para separar as palavras no texto para serem contadas, dependendo da origem do texto pode melhorar muito a análise.
    
    
- ### TfidfVectorizer
  Essa abordagem faz a contagem das palavras por instância(tweets) assim como a CountVectorizer, porém além disso calcula a frequência que essa palavra apareceu baseado em todas as instâncias. Ou seja, uma palavra que aparece muito em um determinado tweet, mas muito pouco nos demais, terá um peso muito maior para a definição da classe daquele tweet. Do contrário, uma palavra que aparece em abundância,  em um tweet, mas é muito comum em todos os outros terá um peso menor.
  
- Hiperparâmetros:
  - max_features:
    Define a quantidade máxima de palavras que será mantido a contagem, no caso o algoritmo sempre priorizará as palavras que mais aparecem, pois, elas têm um maior peso para a definição da classe.
  - analyzer:
    Define se o algoritmo irá analisar palavra como features ou palavras.
  - stop_words:
    Define um conjunto de palavras ou não para ser removido dos textos antes da contagem, geralmente é removido palavras que não são relevantes para a análise.
  - tokenizer:
    Define o critério usado para separar as palavras no texto para serem contadas, dependendo da origem do texto pode melhorar muito a análise
  

# Normalizador selecionado:

- ### TruncatedSVD()
  Geralmente modelos que trabalham com NPL não lidam bem com vetores com uma grande quantidade de zeros seguidos, devido a numerosa quantidade de palavras possíveis dentro do corpus, as instâncias(tweets) não possuirão a maioria das palavras possíveis no corpus, gerando o problema citado acima. Para contornar isso é necessário reduzir para uma dimensão menor esses dados  que já foram filtrados anteriormente na contagem sendo os mais relevantes. Aplicando o redutor de dimensionalidade SVD, esse vetor espaçado com zeros será reduzido.

- Hiperparâmetros:
  - n_components: Define para quantas features o vetor será reduzido.
  

In [9]:
n_splits_cv = 1
n_splits_gs = 5

all_scores = {}

split_cv = ShuffleSplit(n_splits= n_splits_cv, test_size=0.2, random_state = 42)

for corpus_name, corpus_data in corpus.items():

    for model_name, model_data in models.items():

        model_params = {
            f"model__{key}": value for key, value in model_data["hyperparameters"].items()
        }

        for vectorizer_name, vectorizer_data in vectorizers.items():

            vectorize_params = {
                f"vectorizer__{key}": value
                for key, value in vectorizer_data["hyperparameters"].items()
            }

            for normalizer_name, normalizer_data in normalizers.items():

                normalizer_params = {
                    f"normalizer__{key}": value
                    for key, value in normalizer_data["hyperparameters"].items()
                }

                # for scaler_name, scaler_data in scalers.items():

                #     scaler_params = {
                #         f"scaler__{key}": value
                #         for key, value in scaler_data["hyperparameters"].items()
                #     }

                param_distributions = {
                    **model_params,
                    **vectorize_params,
                    **normalizer_params,
                    # **scaler_params,
                }

                pipeline = Pipeline(
                    steps=[
                        ("vectorizer", vectorizer_data["vectorizer_obj"]),
                        ("normalizer", normalizer_data["normalizer_obj"]),
                        # ("scaler", scaler_data["scaler_obj"]),
                        ("model", model_data["model_obj"]),
                    ]
                )

                approach_name = f"{corpus_name}__{model_name}__{vectorizer_name}__{normalizer_name}"

                print(f"Fiting best model to \n{approach_name}", end="\n\n")

                tuned_pipeline = RandomizedSearchCV(
                    pipeline,
                    param_distributions,
                    scoring="f1",
                    cv=n_splits_gs,
                    random_state=42,
                )

                scores = cross_validate(
                    tuned_pipeline,
                    corpus_data["corpus_data"],
                    labels,
                    cv= split_cv,
                    scoring=["accuracy", "f1", "recall"],
                    ramdom_state=42,
                )

                all_scores.update(
                    {
                        approach_name: {
                            "scores": scores,
                            "tuned_pipeline": tuned_pipeline,
                        }
                    }
                )


Fiting best model to 
corpus_raw__KNN__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_raw__KNN__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_raw__SMV__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_raw__SMV__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_raw__GaussianNB__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_raw__GaussianNB__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_steamed__KNN__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_steamed__KNN__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_steamed__SMV__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_steamed__SMV__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_steamed__GaussianNB__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_steamed__GaussianNB__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_lammetized__KNN__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_lammetized__KNN

In [10]:
all_scores


{'corpus_raw__KNN__TfidfVectorizer__TrucatedSVD': {'scores': {'fit_time': array([1.10153365, 0.7823174 ]),
   'score_time': array([0.02262616, 0.01484323]),
   'test_accuracy': array([1.        , 0.90607735]),
   'test_f1': array([1.        , 0.84955752]),
   'test_recall': array([1.        , 0.73846154])},
  'tuned_pipeline': RandomizedSearchCV(cv=2,
                     estimator=Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                                               ('normalizer', TruncatedSVD()),
                                               ('model',
                                                KNeighborsClassifier())]),
                     param_distributions={'model__n_neighbors': [7, 11, 21],
                                          'model__weights': ['uniform',
                                                             'distance'],
                                          'normalizer__n_components': [3, 5, 10,
                                                 

In [24]:
all_scores

corpus_name = "NoThemeTweets"
approach_names = []
fit_times = []
scores_times = []
accuracy_means = []
f1_scores_mean = []
recall_scores_mean = []


for approach_name, score in all_scores.items():
    # print(f"{approach_name}")
    # print(f"{score['scores']}")
    approach_names.append(approach_name)
    fit_times.append(score["scores"]["fit_time"].mean())
    scores_times.append(score["scores"]["score_time"].mean())
    accuracy_means.append(score["scores"]["test_accuracy"].mean())
    f1_scores_mean.append(score["scores"]["test_f1"].mean())
    recall_scores_mean.append(score["scores"]["test_recall"].mean())
    # print("\n")

test_data = data = {
    "approach": approach_names,
    "fit_time": fit_times,
    "score_time": scores_times,
    "accuracy": accuracy_means,
    "f1": f1_scores_mean,
    "recall": recall_scores_mean,
}


test_data_df = pd.DataFrame(test_data)


test_data_df.style.background_gradient()


,approach,fit_time,score_time,accuracy,f1,recall
0,corpus_raw__KNN__TfidfVectorizer__TrucatedSVD,0.941926,0.018735,0.953039,0.924779,0.869231
1,corpus_raw__KNN__CountVectorizer__TrucatedSVD,0.908277,0.025249,0.977901,0.966066,0.958009
2,corpus_raw__SMV__TfidfVectorizer__TrucatedSVD,0.974998,0.021490,0.972376,0.958263,0.941908
3,corpus_raw__SMV__CountVectorizer__TrucatedSVD,5.689691,0.020938,0.983425,0.974675,1.000000
4,corpus_raw__GaussianNB__TfidfVectorizer__TrucatedSVD,1.606019,0.021220,0.950276,0.914185,0.862400
5,corpus_raw__GaussianNB__CountVectorizer__TrucatedSVD,1.973108,0.023699,0.939227,0.899593,0.882822
6,corpus_steamed__KNN__TfidfVectorizer__TrucatedSVD,1.564961,0.019785,0.925414,0.874553,0.839655
7,corpus_steamed__KNN__CountVectorizer__TrucatedSVD,1.700530,0.027762,0.986188,0.977797,0.973528
8,corpus_steamed__SMV__TfidfVectorizer__TrucatedSVD,2.075382,0.024610,0.958564,0.927025,0.907407
9,corpus_steamed__SMV__CountVectorizer__TrucatedSVD,2.036363,0.024188,0.980663,0.970613,0.982845


In [22]:
approach_names = test_data_df.approach

import numpy as np
def get_best_model(x):
    if x.name.endswith("time"):
        return approach_names[np.argmin(x.values)]
    
    return approach_names[np.argmax(x.values)]
    

In [35]:
from statistics import mode
best_approach_name = mode(test_data_df.apply(get_best_model, axis=0).to_list()).split("__")
names = ["corpus", "model", "vectorizer", "normalizer"]

best_approach_dict = {name: value for name , value in zip(names, best_approach_name)}

best_approach_dict

{'corpus': 'corpus_lammetized',
 'model': 'SMV',
 'vectorizer': 'TfidfVectorizer',
 'normalizer': 'TrucatedSVD'}

In [41]:
normalizers

normalizers[best_approach_dict["normalizer"]]["hyperparameters"]


vectorizers[best_approach_dict["vectorizer"]]["vectorizer_obj"]

TfidfVectorizer()

In [ ]:

normalizer_params = {
    f"normalizer__{key}": value
    for key, value in normalizers[best_approach_dict["normalizer"]]["hyperparameters"].items()
}

param_distributions = {
    **model_params,
    **vectorize_params,
    **normalizer_params,
    # **scaler_params,
}

pipeline = Pipeline(
    steps=[
        ("vectorizer",  vectorizers[best_approach_dict["vectorizers"]]["vectorizer_obj"]),
        ("normalizer", normalizers[best_approach_dict["normalizer"]]["hyperparameters"]),
        # ("scaler", scaler_data["scaler_obj"]),
        ("model", model_data["model_obj"]),
    ]
)

approach_name = f"{corpus_name}__{model_name}__{vectorizer_name}__{normalizer_name}"

print(f"Fiting best model to \n{approach_name}", end="\n\n")

tuned_pipeline = RandomizedSearchCV(
    pipeline,
    param_distributions,
    scoring="f1",
    cv=n_splits_gs,
    random_state=42,
)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    df.tweet_text, df.sentiment, test_size=0.2, random_state=42
)

corpus_train2 = x_train.to_list()
labels_train2 = y_train.replace({"Positivo": 1, "Negativo": 0}).to_list()

corpus_test2 = x_test.to_list()
labels_test2 = y_test.replace({"Positivo": 1, "Negativo": 0}).to_list()


all_scores["SMV__TfidfVectorizer__PCA__Scaler"]["tuned_pipeline"].fit(
    corpus_train2, labels_train2
)

print("______" * 30)
print(all_scores["SMV__TfidfVectorizer__PCA__Scaler"]["tuned_pipeline"].best_params_)


In [ ]:
y_hat = all_scores["SMV__TfidfVectorizer__PCA__Scaler"]["tuned_pipeline"].predict(
    corpus_test2
)

y_hat


In [ ]:
accuracy_score(labels_test2, y_hat)


In [ ]:
f1_score(labels_test2, y_hat)
